In [1]:
import gc
import csv
from scipy.misc import imread
import numpy as np
from sklearn.model_selection import train_test_split
import cv2
import sklearn
import json
import matplotlib.pyplot as plt
%matplotlib inline
from keras.models import model_from_json
from keras.models import Sequential
from keras.optimizers import SGD, Adam, RMSprop
from keras.layers import Dense, Dropout, Activation, Flatten, ELU, Lambda
from keras.layers import Convolution2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array
from keras.callbacks import Callback

Using TensorFlow backend.


In [2]:
path = "data/"
filename = "dummy.csv"   
model_name = "dumm"


total = 20
batch_size = 4
epochs = 3
imgsize = (124,256,3)



In [3]:
def load_data(image_paths, steering_angles):
      
    size = len(image_paths)
    print("called load_data", size)
    
    shape = imread(path+image_paths[0]).shape
    global layer_shape
    layer_shape = shape
    
    X = np.zeros(shape=(size,imgsize[0],imgsize[1],imgsize[2]))
    y = np.zeros(shape=(size,1)).astype(float)
    
    for i in range(size):
        img = imread(path+image_paths[i])
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        #img = img[55:140:,:,:]
        
        if(imgsize != shape):#resize if image size is not per your liking
            print("Resizing image")
            img = cv2.resize(img,(imgsize[1],imgsize[0]))

        X[i] = img_to_array(img)
        y[i] = steering_angles[i]
        i+=1
    
    return X, y

def data_generator(batch_size=1):
    print("called data_generator")
    
    image_paths, steering_angles = load_csv()
    #image_paths, steering_angles = shuffle_lists(image_paths, steering_angles) 
    while True:
       for i in range(0,total,batch_size):
           #image_paths, steering_angles = shuffle_lists(image_paths, steering_angles)
           #X,y = load_data(image_paths[i:i+batch_size], steering_angles[i:i+batch_size])
           image_paths_s, steering_angles_s = shuffle_lists(image_paths[i:i+batch_size], steering_angles[i:i+batch_size])
           X,y = load_data(image_paths_s, steering_angles_s)
           print("yeilding y", y)
           print("\n") 
           yield (X, y)
          


In [4]:
def shuffle_lists(X, y):
    
    size = len(X)
    print("called shuffle_lists", size)
    
    X_shuf = []
    y_shuf = []
    
    index_shuf = np.arange(size)
    
    np.random.shuffle(index_shuf)
    
    index = 0
    
    for i in index_shuf:
        X_shuf.append(X[i])
        y_shuf.append(y[i])
        index+=1
    
    return X_shuf, y_shuf 




In [7]:
model = get_model()    
history = model.fit_generator(data_generator(batch_size), samples_per_epoch = total, nb_epoch = epochs,
verbose=2, callbacks=[], validation_data=None, class_weight=None, nb_worker=1)


called get_model
called data_generatorEpoch 1/3

called load_csv
called shuffle_lists 4
called load_data 4
Resizing image
Resizing image
Resizing image
Resizing image
yeilding y [[ 3.]
 [ 4.]
 [ 1.]
 [ 2.]]


called shuffle_lists 4
called load_data 4
Resizing image
Resizing image
Resizing image
Resizing image
yeilding y [[ 7.]
 [ 5.]
 [ 8.]
 [ 6.]]


called shuffle_lists 4
called load_data 4
Resizing image
Resizing image
Resizing image
Resizing image
yeilding y [[ 11.]
 [ 10.]
 [  9.]
 [ 12.]]


called shuffle_lists 4
called load_data 4
Resizing image
Resizing image
Resizing image
Resizing image
yeilding y [[ 13.]
 [ 14.]
 [ 16.]
 [ 15.]]


called shuffle_lists 4
called load_data 4
Resizing image
Resizing image
Resizing image
Resizing image
yeilding y [[ 20.]
 [ 17.]
 [ 19.]
 [ 18.]]


called shuffle_lists 4
called load_data 4
Resizing image
Resizing image
Resizing image
Resizing image
yeilding y [[ 3.]
 [ 2.]
 [ 1.]
 [ 4.]]


called shuffle_lists 4
called load_data 4
Resizing image
Re

In [8]:
image_paths, steering_angles = load_csv()
X_test, y_test = load_data(image_paths[0:4], steering_angles[0:4])

print(model.predict(X_test))

print("Original values...")
for i in range(len(y_test)):
    print(y_test[i])

called load_csv
called load_data 4
Resizing image
Resizing image
Resizing image
Resizing image
[[ 2.99953055]
 [ 2.99554515]
 [ 2.99318171]
 [ 3.00051045]]
Original values...
[ 1.]
[ 2.]
[ 3.]
[ 4.]


In [10]:
import gc
len(gc.get_objects())

248289

In [6]:
def load_csv():
    image_paths= []
    steering_angles = []
    print("called load_csv")
    with open(path+filename, 'r') as f:
        reader = csv.reader(f)
        next(reader, None) 

        index = 0
        for row in reader:
           image_paths.append(row[0])
           steering_angles.append(float(row[3])) 
           index+=1
    return image_paths, steering_angles



In [5]:

def get_model():
    print("called get_model")
    model = Sequential()

    model.add(Lambda(lambda x: x/127.5 - 1., input_shape=imgsize,output_shape=imgsize))

    model.add(Convolution2D(24, 5, 5, border_mode='same', init="normal"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(ELU())

    model.add(Convolution2D(36, 5, 5, border_mode='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(ELU())

    model.add(Convolution2D(48, 5, 5, border_mode='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(ELU())

    model.add(Convolution2D(64, 5, 5, border_mode='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(ELU())


    model.add(Flatten())
    model.add(Dense(100))
    model.add(ELU())

    model.add(Dense(50))
    model.add(ELU())

    model.add(Dense(10))
    model.add(ELU())

    model.add(Dense(1))

    opt = Adam(lr=0.001)
    model.compile(loss='mean_squared_error',optimizer=opt,metrics=['accuracy'])
    
    return model